In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 7
fig_height = 5
fig_format = 'retina'
fig_dpi = 96

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'/Users/zeemarquez/Documents/Python/GithubPages/zeemarquez.github.io/posts/blockchain_1':
  os.chdir(r'/Users/zeemarquez/Documents/Python/GithubPages/zeemarquez.github.io/posts/blockchain_1')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


/var/folders/c4/gwqzwlt13jx3ygr_1zlyc8r00000gn/T/ipykernel_9695/4215672897.py:20: DeprecationWarning:

`set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`



{"/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/importlib/_bootstrap.py": 1582584757.0, "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/importlib/_bootstrap_external.py": 1582584757.0, "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/codecs.py": 1582584756.0, "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/encodings/aliases.py": 1582584758.0, "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/encodings/__init__.py": 1582584758.0, "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/encodings/utf_8.py": 1582584758.0, "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/encodings/latin_1.py": 1582584758.0, "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/abc.py": 1582584756.0, "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/io.py": 1582584756.0, "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/stat.py": 1582584756.0, "/Library/Framewor

In [2]:
import random as rd

print('Rd',rd.random())

Rd 0.3765947714642486


In [3]:
import hashlib as hs
import time

In [4]:
class Transaction:
    def __init__(self, sender, reciever, value):
        self.sender = sender
        self.reciever = reciever
        self.value = value


In [5]:
class Block:
    def __init__(self, timestamp, transactions):
        self.timestamp = timestamp
        self.transactions = transactions
        self.prevHash = None
        self.hash = None
        self.nonce = 0

        
    def calculateHash(self):
        return hs.sha256((''.join([self.timestamp, ''.join(str(x) for x in self.transactions), self.prevHash, str(self.nonce)])).encode()).hexdigest()
    
    def mineBlock(self, difficulty):
        while True:
            self.hash = self.calculateHash()
            if self.hash[0:difficulty] == "".join(["0" for x in range(difficulty)]):
                break
            self.nonce += 1
            
        print("Block mined:",self.hash)

In [6]:
class Blockchain:
    
    sysAddress = "0000"
    
    def __init__(self):
        self.difficulty = 2
        self.miningReward = 100
        self.chain = [self.createGenBlock()]
        self.pendingTransactions = []
        
        
        
    def createGenBlock(self):
        genBlock = Block(str(time.time()),[Transaction(Blockchain.sysAddress,'satoshi',100)])
        genBlock.prevHash = '0'
        genBlock.hash = genBlock.calculateHash()
        return genBlock
    
    def getLastBlock(self):
        return self.chain[-1]
    
    def addBlock(self, newBlock):
        newBlock.prevHash = self.getLastBlock().hash
        newBlock.mineBlock(self.difficulty)
        self.chain.append(newBlock)
    
    def minePending(self, minerAddress):
        self.pendingTransactions.append(Transaction(Blockchain.sysAddress, minerAddress, self.miningReward))
        block = Block(str(time.time()), self.pendingTransactions)
        self.addBlock(block)
        self.pendingTransactions = []
        
    def stageTransaction(self, transaction):
        if self.isTransactionValid(transaction):
            self.pendingTransactions.append(transaction)
        else:
            print("Transaction invalid")
        
    def isValid(self):
        for i in range(1,len(self.chain)):
            
            if self.chain[i-1].hash != self.chain[i].prevHash:
                return False
            
            if self.chain[i].hash != self.chain[i].calculateHash():
                return False
            
        return True
    
    def checkBalance(self, address):
        balance = 0
        for block in self.chain:
            for trans in block.transactions:
                if trans.sender == address:
                    balance -= trans.value
                if trans.reciever == address:
                    balance += trans.value
        return balance
    
    def isTransactionValid(self, transaction):
        if self.checkBalance(transaction.sender) < transaction.value:
            
            return False
        else:
            return True


In [7]:
zcoin = Blockchain()

zcoin.minePending('alice')

zcoin.stageTransaction(Transaction('alice', 'bob', 25))

zcoin.minePending('bob')

print('\nBalance Alice:', zcoin.checkBalance('alice'))
print('Balance Bob:', zcoin.checkBalance('bob'))

Block mined: 004ffff9e754f5c7ce6e61641dc6d099a97f9614229bb88aaabb3dacf944e5d1
Block mined: 00332f749020236fb7cc81fcfa85ff7a6eb9f8337878d130a46b9185ddc877bf

Balance Alice: 75
Balance Bob: 125


In [8]:
zcoin.isValid()

True

In [9]:
from ecies import utils
from ecies import encrypt, decrypt
import ecies 

def genKeyPair():
    private_key = utils.generate_key()
    public_key = private_key.public_key
    return (private_key.to_hex(),public_key.format().hex())

def sign(data, signingKey):
    k = utils.generate_key().from_hex(signingKey)
    return k.sign(data.encode())
    
def verify(data, signature, publicKey):
    try:
        kpub = ecies.hex2pub(publicKey)
    except:
        return False
    return kpub.verify(signature, data.encode())
    
    
def getPublicKey(private_key):
    k = utils.generate_key().from_hex(private_key)
    return k.public_key.format().hex()


In [10]:
private_key, public_key = genKeyPair()

print('Private: ', private_key)
print('Public: ', public_key)

Private:  bb0ad2177977aa3c3184cb255748de975fda676759640d2d7f838ee18079b65f
Public:  03e082c7b1ed1a5f56261d087ccb0998e2c103be2482ced2abd14fd30b244d043d


In [11]:
sig = sign('message', private_key)

print(verify('message', sig, public_key))

print(getPublicKey(private_key))


True
03e082c7b1ed1a5f56261d087ccb0998e2c103be2482ced2abd14fd30b244d043d


In [12]:
class Transaction:
    def __init__(self, sender, reciever, value):
        self.sender = sender
        self.reciever = reciever
        self.value = value
        self.signature = None
        
    def calculateHash(self):
        
        return hs.sha256((''.join([self.sender, self.reciever, str(self.value)])).encode()).hexdigest()
    
    def signTransaction(self, signKey):
        
        if getPublicKey(signKey) != self.sender:
            print("You cannot sign transactions for other wallets!")
            
        hashTx = self.calculateHash()
        self.signature =  sign(hashTx, signKey)
        
    def isValid(self):
        
        if self.sender == Blockchain.sysAddress:
            return True
        
        if self.signature == None:
            return False
        
        public_key = getPublicKey(self.sender)
        
        return verify(self.calculateHash(), self.signature, public_key)


In [13]:
class Block:
    def __init__(self, timestamp, transactions):
        self.timestamp = timestamp
        self.transactions = transactions
        self.prevHash = None
        self.hash = None
        self.nonce = 0

    def calculateHash(self):
        return hs.sha256((''.join([self.timestamp, str(self.transactions), self.prevHash, str(self.nonce)])).encode()).hexdigest()
    
    def mineBlock(self, difficulty):
        while True:
            self.hash = self.calculateHash()
            if self.hash[0:difficulty] == "".join(["0" for x in range(difficulty)]):
                break
            self.nonce += 1
            
        print("Block mined:",self.hash)
        
    def checkValidTransactions(self):
        for tx in self.transactions:
            if not tx.isValid():
                return False
        return True
    
    def __str__(self):
        return '\n'.join([ '|' + key + '|\t' + self.__dict__[key].__str__() +'|' for key in self.__dict__ ]) 

In [14]:
class Blockchain:
    
    sysAddress = "0000"
    
    def __init__(self):
        self.difficulty = 2
        self.miningReward = 100
        self.chain = [self.createGenBlock()]
        self.pendingTransactions = []
        
    def createGenBlock(self):
        genBlock = Block(str(time.time()),[Transaction(Blockchain.sysAddress,'satoshi',100)])
        genBlock.prevHash = '0'
        genBlock.hash = genBlock.calculateHash()
        return genBlock
    
    def getLastBlock(self):
        return self.chain[-1]
    
    def addBlock(self, newBlock):
        newBlock.prevHash = self.getLastBlock().hash
        newBlock.mineBlock(self.difficulty)
        self.chain.append(newBlock)
    
    def minePending(self, minerAddress):
        self.pendingTransactions.append(Transaction(Blockchain.sysAddress, minerAddress, self.miningReward))
        block = Block(str(time.time()), self.pendingTransactions)
        self.addBlock(block)
        self.pendingTransactions = []
        
    def stageTransaction(self, transaction):
        if self.isTransactionValid(transaction):
            self.pendingTransactions.append(transaction)
        else:
            raise Exception("Transaction invalid")
        
    def isValid(self):
        for i in range(1,len(self.chain)):
            
            if self.chain[i-1].hash != self.chain[i].prevHash:
                return False
            
            if self.chain[i].hash != self.chain[i].calculateHash():
                return False
            
            #Added check of valid transactions
            if not self.chain[i].checkValidTransactions():
                return False
            
        return True
    
    def checkBalance(self, address):
        balance = 0
        for block in self.chain:
            for trans in block.transactions:
                if trans.sender == address:
                    balance -= trans.value
                if trans.reciever == address:
                    balance += trans.value
        return balance
    
    def isTransactionValid(self, transaction):
        if self.checkBalance(transaction.sender) < transaction.value:
            return False
        else:
            return True


In [15]:
myKey, myWalletAddress = genKeyPair()

AliceKey, AliceWalletAddress = genKeyPair()
BobKey, BobWalletAddress = genKeyPair()

del AliceKey, BobKey

print('My address:', myWalletAddress)
print('Alice address:', AliceWalletAddress)
print('Bob address:', BobWalletAddress)

My address: 02d604b41721e24bb2e588689057a17c580d3352eb059d3a51bafe48d6638aaa73
Alice address: 02844e6a0a04be94882967b19cdcb9976b775eea1c7653bb5fef6a2e6239e4c033
Bob address: 03836cc89aa645a2e01fc5c4f463c48c062ecf55b624e763223e13285fee4fb068


In [16]:
zcoin = Blockchain()

zcoin.minePending(myWalletAddress)
print('My balance:',zcoin.checkBalance(myWalletAddress))

Block mined: 00fe4874a306f64ec1aeadb56a1d7a0c7f94a4cae232a07bb893072433cc00f9
My balance: 100


In [17]:
tx1 = Transaction(myWalletAddress, AliceWalletAddress, 50)
tx2 = Transaction(myWalletAddress, BobWalletAddress, 25)

tx1.signTransaction(myKey)
tx2.signTransaction(myKey)

zcoin.stageTransaction(tx1)
zcoin.stageTransaction(tx2)

zcoin.minePending(AliceWalletAddress)

print('My balance:',zcoin.checkBalance(myWalletAddress))

Block mined: 00be4fb6068ab7c29daa2d12fe4b7311b6c28582dc894d6428a97104eed92048
My balance: 25
